In [ ]:
!pip install --upgrade google-cloud-aiplatform

In [ ]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

import vertexai
from vertexai.generative_models import GenerativeModel
from vertexai.preview.tuning import sft

vertexai.init(project="amplified-alpha-437112-f0", location="europe-west4")

gemini_pro = GenerativeModel("gemini-1.0-pro-002")

sft_tuning_job = sft.train(
    source_model=gemini_pro,
    train_dataset="gs://0930_test_first_bucket/teat_google_data/sft_train_data.jsonl",
    validation_dataset="gs://0930_test_first_bucket/teat_google_data/stf_validate.jsonl",
    tuned_model_display_name="0930_2nd_attempt_with_sample_data",
    epochs=2,
    learning_rate_multiplier=1,
)

# Formatting

In [ ]:
# read the csv file
import json
import pandas as pd

df_mqm = pd.read_csv('data/0929_redebug.csv', index_col='Unnamed: 0')

# convert it to the gcp accepted jsonl format
def output_gcp_jsonl(src, tgt,qs):
    output =  {"contents": [{"role": "user", "parts": [{"text": f"source: {src} \n target: {tgt}"}]}, {"role": "model", "parts": [{"text": str(qs)}]}]}
    return output

df_mqm['gcp_format'] = df_mqm.apply(lambda row: output_gcp_jsonl(row['source'], row['target'], row['quality_score']), axis = 1)
my_output_list_for_gcp = list(df_mqm['gcp_format'])

output_path = 'data/gcp_output_mqm_test.jsonl'
# write the file
with open(output_path, 'w',encoding='utf-8') as f:
    for item in my_output_list_for_gcp:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')